In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='7'

In [ ]:
from huggingface_hub import login

login('Your Token')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer,  Trainer, TrainingArguments

# Load the FLAN-T5 model and tokenizer
model_name =  "soumagok/flan-t5-base-cnn_dailymail" # You can choose other sizes like small, base, xl
model_name =  "google/flan-t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
import pandas as pd
from datasets import Dataset

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
eval_df = pd.read_csv("eval.csv")

# Filter DataFrames to retain only the required columns
train_filtered = train_df[["transcripts", "human_summaries"]]
test_filtered = test_df[["transcripts", "human_summaries"]]
eval_filtered = eval_df[["transcripts", "human_summaries"]]

# Convert the transformed data into Hugging Face datasets
train_dataset = Dataset.from_pandas(train_filtered)
test_dataset = Dataset.from_pandas(test_filtered)
eval_dataset = Dataset.from_pandas(eval_filtered)

In [ ]:
prefix = "Summarize the given text: "
def preprocess_function(examples):
    # Add prefix to inputs
    inputs = [prefix + doc for doc in examples["transcripts"]]

    # Tokenize inputs with padding and truncation
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    # Tokenize labels with padding and truncation
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["human_summaries"], max_length=128, truncation=True, padding="max_length")

    # Assign tokenized labels to model_inputs
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs
    return model_inputs
tokenized_train_data = train_dataset.map(preprocess_function, batched=True)
tokenized_test_data = test_dataset.map(preprocess_function, batched=True)
tokenized_eval_data =  eval_dataset.map(preprocess_function, batched=True)

In [ ]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="flan_no",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    #save_steps=500,
    #logging_dir="./logs",
    logging_steps=10,
    push_to_hub=False,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_eval_data,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# # Save the model
# model.save_pretrained("./fine-tuned-bart-summarization")
# tokenizer.save_pretrained("./fine-tuned-bart-summarization")

# # Evaluate the model
# metric = load_metric("rouge")

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     return {k: round(v, 4) for k, v in result.items()}

# trainer.evaluate(compute_metrics=compute_metrics)

In [ ]:
# trainer.push_to_hub()

In [ ]:
model.save_pretrained("./flan_no_sum")
tokenizer.save_pretrained("./flan_no_sum")

('./flan_no_sum/tokenizer_config.json',
 './flan_no_sum/special_tokens_map.json',
 './flan_no_sum/spiece.model',
 './flan_no_sum/added_tokens.json')